In [1]:
import numpy as np
from sklearn.decomposition import PCA
import scipy.io
import itertools
import os
import h5py
import pyret.visualizations as pyviz
import pyret.filtertools as ft
import pyret.spiketools as st
import pyret.stimulustools as stimtools
from pyret.spiketools import binspikes
import jetpack
from jetpack.timepiece import hrtime
from experiments.iotools import read_channel # from niru-analysis github
from experiments.photodiode import find_peaks, find_start_times
# import binary     # in igor >> recording
import pdb
import string
# from jetpack.signals import peakdet
from scipy.signal import find_peaks_cwt
from os.path import expanduser

# This is a bit of magic to make matplotlib figures appear inline in the
# notebook rather than in a new window.
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, xlabel, ylabel, title, imshow

# note that nonposx(y) for log plots will no longer work with this package
import mpld3
#mpld3.enable_notebook()

from pylab import rcParams
rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

You must install GPy (pip install GPy) to fit the GP regression nonlinearity.


# experiment directory

In [4]:
# data_dir = '/Users/lmcintosh/experiments/data/16-05-31/'
# stim_dir = '/Users/lmcintosh/experiments/data/16-05-31/stimulus/'
data_dir = '/Volumes/group/baccus/Lane/2016-05-31/'
stim_dir = '/Volumes/group/baccus/Lane/2016-05-31/stimulus/'
stimulus_filename = 'stimulus.h5'

# load spikes

In [5]:
fs = sorted(os.listdir(data_dir))
fs = [f for f in fs if f.endswith(".txt")]

cells = []
for f in fs:
    text_file = open(data_dir + f, "r")
    spikes = text_file.read().split('\n')
    cells.append([float(spike) for spike in spikes if (not (not spike)) and float(spike) > 0])
    
    text_file.close()

In [6]:
len(cells)

28

In [7]:
f = h5py.File(stim_dir + stimulus_filename)
print(len(list(f)))
list(f)

128


['expt1',
 'expt10',
 'expt100',
 'expt101',
 'expt102',
 'expt103',
 'expt104',
 'expt105',
 'expt106',
 'expt107',
 'expt108',
 'expt109',
 'expt11',
 'expt110',
 'expt111',
 'expt112',
 'expt113',
 'expt114',
 'expt115',
 'expt116',
 'expt117',
 'expt118',
 'expt119',
 'expt12',
 'expt120',
 'expt121',
 'expt122',
 'expt123',
 'expt124',
 'expt125',
 'expt126',
 'expt127',
 'expt128',
 'expt13',
 'expt14',
 'expt15',
 'expt16',
 'expt17',
 'expt18',
 'expt19',
 'expt2',
 'expt20',
 'expt21',
 'expt22',
 'expt23',
 'expt24',
 'expt25',
 'expt26',
 'expt27',
 'expt28',
 'expt29',
 'expt3',
 'expt30',
 'expt31',
 'expt32',
 'expt33',
 'expt34',
 'expt35',
 'expt36',
 'expt37',
 'expt38',
 'expt39',
 'expt4',
 'expt40',
 'expt41',
 'expt42',
 'expt43',
 'expt44',
 'expt45',
 'expt46',
 'expt47',
 'expt48',
 'expt49',
 'expt5',
 'expt50',
 'expt51',
 'expt52',
 'expt53',
 'expt54',
 'expt55',
 'expt56',
 'expt57',
 'expt58',
 'expt59',
 'expt6',
 'expt60',
 'expt61',
 'expt62',
 'expt63'

In [ ]:
raw_data = h5py.File(data_dir + '16-05-31.h5')
pd = read_channel(raw_data, channel=119)

In [ ]:
uniques = np.unique(pd)
time = np.linspace(0, len(pd)/20000., len(pd))
timestamps = [time[i] for i,p in enumerate(pd) if p == uniques[0]]

In [ ]:
peaks = [t for i,t in enumerate(timestamps[1:]) if t > (timestamps[i]+5.0)]
peaks.insert(0, timestamps[0])

# Align spikes within the experiment blocks

In [ ]:
start_times = peaks

In [ ]:
expt_spikes = []
for expt_id, start in enumerate(start_times):
    expt_spikes.append([])
    expt_name = 'expt%d' %(expt_id+1)
    stop = f[expt_name + '/timestamps'][-1] # last timestamp, in seconds
    for c in cells:
        expt_spikes[-1].append([sp-start for sp in c if sp >= start and sp <= start+stop])

In [ ]:
def spikes_to_array(spikes):
    '''
    Converts a list of spike times to an array with labels for easy
    pyret.visualizations.raster() use.
    
    INPUT:
        spikes    list of list of spike times
    '''
    collapsed_spikes = []
    collapsed_labels = []
    nlists = len(spikes)
    for idl, l in enumerate(spikes):
        labels = idl * np.ones((len(l),))
        collapsed_spikes.extend(l)
        collapsed_labels.extend(labels)
        
    return np.array(collapsed_spikes), np.array(collapsed_labels)
    

In [ ]:
all_spikes, all_labels = spikes_to_array(expt_spikes[1])

In [ ]:
stimulus_seq = []
stimulus_seq.append('whitenoise')
# 14 single trials of 5 min each (70 min total)
for block in range(14):
    stimulus_seq.append('naturalscene_single')
    # 14 * 8 = 112 repeats
    for repeat in range(8):
        stimulus_seq.append('naturalscene_repeat')
stimulus_seq.append('whitenoise')

In [ ]:
whitenoise_test_expts = [i for i,stim_type in enumerate(stimulus_seq) if stim_type == 'whitenoise']
naturalscene_train_expts = [i for i,stim_type in enumerate(stimulus_seq) if stim_type == 'naturalscene_single']
naturalscene_test_expts = [i for i,stim_type in enumerate(stimulus_seq) if stim_type == 'naturalscene_repeat']
print('# whitenoise = %i, # ns repeats = %i, # ns single trial = %i' %(len(whitenoise_test_expts),
                                                                      len(naturalscene_test_expts),
                                                                      len(naturalscene_train_expts)))

# Save files

In [20]:
with h5py.File(stim_dir + stimulus_filename) as f:
    with h5py.File('/Users/lmcintosh/experiments/data/16-05-31/naturalscene.h5', 'w') as h:

        ##### spikes #####
        ncells = len(cells)
        for idx in range(ncells):
            h.create_dataset('spikes/cell%02d' %(idx+1), data=np.array(cells[idx]))

        ##### train #####
        train_stimuli = []
        train_times = []
        train_response_binned = []
        train_response_5ms = []
        train_response_10ms = []
        train_response_20ms = []
        last_time = 0.0
        for idx in naturalscene_train_expts:
            expt_name = 'expt%d' %(idx+1)
            stim = f[expt_name + '/stim']
        #     time = f[expt_name + '/timestamps']
            time = np.arange(0, stim.shape[0])/25.00940617
            stimulus_upsample, time_upsample = stimtools.upsample_stim(stim, 4, time)
            binned_spikes = []
            response_5ms = []
            response_10ms = []
            response_20ms = []
            for c in range(len(cells)):
                bspk, tax = binspikes(expt_spikes[idx][c], binsize=0.01, time=time_upsample)
                binned_spikes.append(bspk)
                r5ms = st.estfr(tax, bspk, sigma=0.005) # 5 ms Gaussian
                r10ms = st.estfr(tax, bspk, sigma=0.01) # 10 ms Gaussian
                r20ms = st.estfr(tax, bspk, sigma=0.02) # 20 ms Gaussian
                response_5ms.append(r5ms)
                response_10ms.append(r10ms)
                response_20ms.append(r20ms)
            train_response_binned.append(np.vstack(binned_spikes))
            train_response_5ms.append(np.vstack(response_5ms))
            train_response_10ms.append(np.vstack(response_10ms))
            train_response_20ms.append(np.vstack(response_20ms))
            train_stimuli.append(stimulus_upsample[:-1])
            train_times.append(time_upsample[:-1] + last_time)
            last_time += time_upsample[-1]
        # create datasets
        h.create_dataset('train/stimulus', data=np.vstack(train_stimuli))
        h.create_dataset('train/time', data=np.hstack(train_times))
        h.create_dataset('train/response/binned', data=np.hstack(train_response_binned))
        h.create_dataset('train/response/firing_rate_10ms', data=np.hstack(train_response_10ms))
        h.create_dataset('train/response/firing_rate_20ms', data=np.hstack(train_response_20ms))
        h.create_dataset('train/response/firing_rate_5ms', data=np.hstack(train_response_5ms))

        ##### test #####
        test_stimuli = []
        test_times = []
        test_response_binned = []
        test_response_5ms = []
        test_response_10ms = []
        test_response_20ms = []
        test_repeats = []
        for repeat, idx in enumerate(naturalscene_test_expts):
            expt_name = 'expt%d' %(idx+1)
            stim = f[expt_name + '/stim']
        #     time = f[expt_name + '/timestamps']
            time = np.arange(0, stim.shape[0])/25.00940617
            stimulus_upsample, time_upsample = stimtools.upsample_stim(stim, 4, time)
            binned_spikes = []
            response_5ms = []
            response_10ms = []
            response_20ms = []
            for c in range(len(cells)):
                bspk, tax = binspikes(expt_spikes[idx][c], binsize=0.01, time=time_upsample)
                binned_spikes.append(bspk)
                r5ms = st.estfr(tax, bspk, sigma=0.005) # 5 ms Gaussian
                r10ms = st.estfr(tax, bspk, sigma=0.01) # 10 ms Gaussian
                r20ms = st.estfr(tax, bspk, sigma=0.02) # 20 ms Gaussian
                response_5ms.append(r5ms)
                response_10ms.append(r10ms)
                response_20ms.append(r20ms)
                if repeat == 0:
                    test_repeats.append([bspk])
                else:
                    test_repeats[c].append(bspk)
            test_response_binned.append(np.vstack(binned_spikes))
            test_response_5ms.append(np.vstack(response_5ms))
            test_response_10ms.append(np.vstack(response_10ms))
            test_response_20ms.append(np.vstack(response_20ms))
        # create datasets
        h.create_dataset('test/stimulus', data=stimulus_upsample[:-1])
        h.create_dataset('test/time', data=time_upsample[:-1])
        h.create_dataset('test/response/binned', data=np.mean(test_response_binned, axis=0))
        h.create_dataset('test/response/firing_rate_10ms', data=np.mean(test_response_10ms, axis=0))
        h.create_dataset('test/response/firing_rate_20ms', data=np.mean(test_response_20ms, axis=0))
        h.create_dataset('test/response/firing_rate_5ms', data=np.mean(test_response_5ms, axis=0))
        for c in range(len(cells)):
            dataset_name = 'test/repeats/cell%02d' %(c+1)
            h.create_dataset(dataset_name, data=np.vstack(test_repeats[c]))

        h.close()

/Users/lmcintosh/code/pyret/pyret/spiketools.py:141: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return np.convolve(filt, bspk, mode='full')[size:size + tax.size] / dt


In [ ]:
with h5py.File(stim_dir + stimulus_filename) as f:
    with h5py.File('/Users/lmcintosh/experiments/data/16-05-31/whitenoise.h5', 'w') as h:

        ##### spikes #####
        ncells = len(cells)
        for idx in range(ncells):
            h.create_dataset('spikes/cell%02d' %(idx+1), data=np.array(cells[idx]))

        ##### test #####
        test_stimuli = []
        test_times = []
        test_response_binned = []
        test_response_5ms = []
        test_response_10ms = []
        test_response_20ms = []
        test_repeats = []
        test_spike_times = []
        for repeat, idx in enumerate(whitenoise_test_expts):
            expt_name = 'expt%d' %(idx+1)
            stim = f[expt_name + '/stim']
        #     time = f[expt_name + '/timestamps']
            time = np.arange(0, stim.shape[0])/25.00940617
            stimulus_upsample, time_upsample = stimtools.upsample_stim(stim, 4, time)
            binned_spikes = []
            spike_times = []
            response_5ms = []
            response_10ms = []
            response_20ms = []
            for c in range(len(cells)):
                bspk, tax = binspikes(expt_spikes[idx][c], binsize=0.01, time=time_upsample)
                binned_spikes.append(bspk)
                r5ms = st.estfr(tax, bspk, sigma=0.005) # 5 ms Gaussian
                r10ms = st.estfr(tax, bspk, sigma=0.01) # 10 ms Gaussian
                r20ms = st.estfr(tax, bspk, sigma=0.02) # 20 ms Gaussian
                response_5ms.append(r5ms)
                response_10ms.append(r10ms)
                response_20ms.append(r20ms)
                if repeat == 0:
                    test_repeats.append([bspk])
                else:
                    test_repeats[c].append(bspk)
            test_response_binned.append(np.vstack(binned_spikes))
            test_response_5ms.append(np.vstack(response_5ms))
            test_response_10ms.append(np.vstack(response_10ms))
            test_response_20ms.append(np.vstack(response_20ms))
        # create datasets
        h.create_dataset('test/stimulus', data=stimulus_upsample[:-1])
        h.create_dataset('test/time', data=time_upsample[:-1])
        h.create_dataset('test/response/binned', data=np.mean(test_response_binned, axis=0))
        h.create_dataset('test/response/firing_rate_10ms', data=np.mean(test_response_10ms, axis=0))
        h.create_dataset('test/response/firing_rate_20ms', data=np.mean(test_response_20ms, axis=0))
        h.create_dataset('test/response/firing_rate_5ms', data=np.mean(test_response_5ms, axis=0))
        for c in range(len(cells)):
            dataset_name = 'test/repeats/cell%02d' %(c+1)
            h.create_dataset(dataset_name, data=np.vstack(test_repeats[c]))

        h.close()

### Going to make a compressed version

In [ ]:
with h5py.File(stim_dir + stimulus_filename) as f:
    with h5py.File('/Users/lmcintosh/experiments/data/16-05-31/whitenoise_compressed.h5', 'w') as h:
        ##### test #####
        test_stimuli = []
        test_times = []
        test_response_10ms = []
        test_repeats = []
        test_spike_times = []
        for repeat, idx in enumerate(whitenoise_test_expts):
            expt_name = 'expt%d' %(idx+1)
            stim = f[expt_name + '/stim']
        #     time = f[expt_name + '/timestamps']
            time = np.arange(0, stim.shape[0])/25.00940617
            stimulus_upsample, time_upsample = stimtools.upsample_stim(stim, 4, time)
            binned_spikes = []
            spike_times = []
            response_10ms = []
            for c in range(len(cells)):
                bspk, tax = binspikes(expt_spikes[idx][c], binsize=0.01, time=time_upsample)
                binned_spikes.append(bspk)
                r10ms = st.estfr(tax, bspk, sigma=0.01) # 10 ms Gaussian
                response_10ms.append(r10ms)
                if repeat == 0:
                    test_repeats.append([bspk])
                else:
                    test_repeats[c].append(bspk)
            test_response_binned.append(np.vstack(binned_spikes))
            test_response_10ms.append(np.vstack(response_10ms))
        # create datasets
        h.create_dataset('test/stimulus', data=stimulus_upsample[:-1])
        h.create_dataset('test/time', data=time_upsample[:-1])
        h.create_dataset('test/response/firing_rate_10ms', data=np.mean(test_response_10ms, axis=0))
        for c in range(len(cells)):
            dataset_name = 'test/repeats/cell%02d' %(c+1)
            h.create_dataset(dataset_name, data=np.vstack(test_repeats[c]).astype('uint8'))

        h.close()